In [2]:
import pandas as pd

<b> Introduction </b>

In combat sports, we often hear the terms height, reach and weight advantage.
It is generally taken as a given that these advantages are by and large, advantages.  

It has also been noted in various sports that proportions are relevant as well as absolute measures.  In MMA, many people are aware of what can be described as the ape-index.  This compares the fighter's reach (length from fingertip to fingertip with arms outstretched) to their height.  A notable example is the widely considered GOAT (greatest of all time) Jon Jones who is 6'4 (193cm) tall with a reach of 7'0 (213cm). For comparison, a typical man will have a reach that is equal to his height.

This project attempts to quantify the extent of these advantages in MMA, if they exist at all.
The data is limited to the UFC which is generally consired to be the top tier organization of MMA. 

Note: It must be considered that this data mostly looks at fights which are restricted to weight classes. Therefore, the differences between fighters are generally small.

To do list

Ape index [x]
Reach [x]
Height [x]
Weight [x]

(same as above but restricted to heavyweight because of weight differences)

fight wins, streak, longest streak, [x]

champ correlations []
dividing nicely []
adding graphics[]



In [80]:
df=pd.read_csv("C:/Users/dariu/Desktop/Kaggle/ufc-master.csv")

####Converting winner to numeric to allow correlation
df["Red_wins"]=df["Winner"].apply(lambda x: 1 if x == "Red" else -1)

In [110]:
###Height

df["R_Height_adv"]=df["R_Height_cms"]-df["B_Height_cms"]
df_height= df[["R_Height_cms","R_Height_adv", "Red_wins","R_longest_win_streak", "R_current_win_streak"]]
df_height.corr()

,R_Height_cms,R_Height_adv,Red_wins,R_longest_win_streak,R_current_win_streak
R_Height_cms,1.000000,0.380879,-0.005663,0.110969,0.024510
R_Height_adv,0.380879,1.000000,0.034672,-0.001862,0.022734
Red_wins,-0.005663,0.034672,1.000000,0.028224,0.062771
R_longest_win_streak,0.110969,-0.001862,0.028224,1.000000,0.512995
R_current_win_streak,0.024510,0.022734,0.062771,0.512995,1.000000


In [112]:
#### Weight and height at heavyweight
### Weight differences only recorded at heavyweight. All other fighters weigh in at upper limit.

df["R_Weight_adv"]=df["R_Weight_lbs"]-df["B_Weight_lbs"]
df_heavyweight=df[df["weight_class"] == "Heavyweight"]
df_weight= df_heavyweight[["R_Weight_lbs","R_Weight_adv", "R_Height_adv", "Red_wins","R_longest_win_streak", "R_current_win_streak","R_longest_win_streak", "R_current_win_streak"]]
df_weight.corr()




,R_Weight_lbs,R_Weight_adv,R_Height_adv,Red_wins,R_longest_win_streak,R_current_win_streak,R_longest_win_streak,R_current_win_streak
R_Weight_lbs,1.000000,0.628967,0.146651,0.029767,-0.039970,-0.019712,-0.039970,-0.019712
R_Weight_adv,0.628967,1.000000,0.190830,0.097206,-0.062694,0.008019,-0.062694,0.008019
R_Height_adv,0.146651,0.190830,1.000000,0.089095,0.112209,0.068256,0.112209,0.068256
Red_wins,0.029767,0.097206,0.089095,1.000000,0.006986,-0.034413,0.006986,-0.034413
R_longest_win_streak,-0.039970,-0.062694,0.112209,0.006986,1.000000,0.385404,1.000000,0.385404
R_current_win_streak,-0.019712,0.008019,0.068256,-0.034413,0.385404,1.000000,0.385404,1.000000
R_longest_win_streak,-0.039970,-0.062694,0.112209,0.006986,1.000000,0.385404,1.000000,0.385404
R_current_win_streak,-0.019712,0.008019,0.068256,-0.034413,0.385404,1.000000,0.385404,1.000000


In [83]:
####Reach

df["R_Reach_adv"] = df["reach_dif"]
df_reach=df[["R_Reach_cms", "B_Reach_cms","R_Reach_adv","Red_wins","R_longest_win_streak", "R_current_win_streak"]]

df_reach.corr()


,R_Reach_cms,B_Reach_cms,R_Reach_adv,Red_wins,R_longest_win_streak,R_current_win_streak
R_Reach_cms,1.000000,0.668630,-0.359982,0.012622,0.143428,0.058710
B_Reach_cms,0.668630,1.000000,0.396651,-0.045491,0.117344,0.018184
R_Reach_adv,-0.359982,0.396651,1.000000,-0.072587,-0.019035,-0.040571
Red_wins,0.012622,-0.045491,-0.072587,1.000000,0.028224,0.062771
R_longest_win_streak,0.143428,0.117344,-0.019035,0.028224,1.000000,0.512995
R_current_win_streak,0.058710,0.018184,-0.040571,0.062771,0.512995,1.000000


In [119]:
### Is the Ape Index a significant factor? (having long arms vs height) ###

#####Creating columns that generate fighter's ape indices (ratio of height to reach).
#### "AI diff" represents the difference between the two fighters.

df["R_ape_index"]=df["R_Height_cms"]/df["R_Reach_cms"]
df["B_ape_index"]=df["B_Height_cms"]/df["B_Reach_cms"]
df["AI_diff"]=df["R_ape_index"]/df["B_ape_index"]

###Creating smaller data frame
df_ape=df[["AI_diff","R_ape_index","Red_wins","R_longest_win_streak", "R_current_win_streak"]]

df_ape.corr()

,AI_diff,R_ape_index,Red_wins,R_longest_win_streak,R_current_win_streak
AI_diff,1.000000,0.619088,-0.055090,-0.037057,-0.036473
R_ape_index,0.619088,1.000000,-0.036608,-0.108291,-0.079143
Red_wins,-0.055090,-0.036608,1.000000,0.028224,0.062771
R_longest_win_streak,-0.037057,-0.108291,0.028224,1.000000,0.512995
R_current_win_streak,-0.036473,-0.079143,0.062771,0.512995,1.000000


In [118]:
###How do rankings affect odds and fight outcomes?
#### Note that lower rank is good.  Therefore negative correlations with positive results should be expected.

df["red_rank_advantage"]=df["R_match_weightclass_rank"]-df["B_match_weightclass_rank"]
df_rank_favourite=df[["red_rank_advantage","R_odds", "Red_wins"]]
df_rank_favourite.corr()

,red_rank_advantage,R_odds,Red_wins
red_rank_advantage,1.000000,0.258064,-0.083860
R_odds,0.258064,1.000000,-0.332043
Red_wins,-0.083860,-0.332043,1.000000


In [ ]:
<b> Conclusion </b>

It seems that 